In [ ]:
#This solution the methylation calls file from bismark as the input instead of the bedgraph file

In [1]:
#count the number of methylated/non-methylated sites for each CpG site
SRR3083026_CpG_file = "/Users/cmdb/qbb2020-answers/assignment6/CpG_context_SRR3083926_1.chr6_bismark_bt2_pe.txt"
SRR3083029_CpG_file = "/Users/cmdb/qbb2020-answers/assignment6/CpG_context_SRR3083929_1.chr6_bismark_bt2_pe.txt"

#bismark_methylation_extractor format
'''
(1) seq-ID
(2) methylation state, + for methylated, - for unmethylated
(3) chromosome
(4) start position (= end position)
(5) methylation call
'''

#save methyl calls where site bp in chr6 is the key, with elements as two element list
#first element of list is # of methyl calls, second is # of unmethylated calls
SRR3083026_CpG_calls = {}

SRR3083029_CpG_calls = {}

#parse SRR3083026_CpG_file 
with open(SRR3083026_CpG_file, 'r') as f:
    for line in f:
        #skip first line header
        if "Bismark" in line:
            continue
        fields = line.rstrip().split('\t')
        methyl_call = fields[1]
        site = fields [3]
        if site not in SRR3083026_CpG_calls:
            SRR3083026_CpG_calls[site] = [0,0]
        if methyl_call == "+":
            SRR3083026_CpG_calls[site][0] +=1
        else: 
            SRR3083026_CpG_calls[site][1] +=1        
        

#parse SRR3083029_CpG_file
with open(SRR3083029_CpG_file, 'r') as f:
    for line in f:
        #skip first line header
        if "Bismark" in line:
            continue
        fields = line.rstrip().split('\t')
        methyl_call = fields[1]
        site = fields [3]
        if site not in SRR3083029_CpG_calls:
            SRR3083029_CpG_calls[site] = [0,0]
        if methyl_call == "+":
            SRR3083029_CpG_calls[site][0] +=1
        else: 
            SRR3083029_CpG_calls[site][1] +=1  


In [2]:
#filter out redundant gene coordinates from chr6 bed file
#5th and 6th columns are gene coordinates while the 13th column is the gene

input_bedfile = "/Users/cmdb/qbb2020-answers/assignment6/mm10_refseq_genes_chr6_50M_60M.bed"
filtered_bedfile = "/Users/cmdb/qbb2020-answers/assignment6/mm10_refseq_genes_chr6_50M_60M_filtered.bed"

#keep track of unique genes detected
#use dictionary as a hash table for look-up instead of a list which will be slower

genes = {}
output = open(filtered_bedfile, 'w')
with open(input_bedfile, 'r') as f:
    for line in f: 
        fields = line.rstrip().split('\t')
        gene = fields[12]
        gene_start = fields[4]
        gene_end = fields[5]
        if gene not in genes:
            genes[gene] = [gene_start, gene_end]
            output.write(line)
        
output.close()



In [3]:
#convert dictionary to list of tuples (to allow sorting), where first element of tuple is the gene name
#second element is gene start location, and the third element is gene end location
gene_loci = []
gene_keys = genes.keys()
for key in gene_keys:
    gene_location = genes[key]
    gene_locus = (key, gene_location[0], gene_location[1])
    gene_loci.append(gene_locus)


#sort the gene list. This should technically be the same as the gene_loci list since the original bed file looked sorted
#this is just in-case the gene bed file was not sorted
sorted_gene_loci = sorted(gene_loci, key=lambda locus: locus[1])

In [4]:
#save methyl calls where site bp in genes are the keys, with elements as two element list
#first element of list is # of methyl calls, second is # of unmethylated calls
SRR3083026_gene_methylcounts = {}
for loci in sorted_gene_loci:
    gene_name = loci[0]
    SRR3083026_gene_methylcounts[gene_name] = [0,0]


methyl_positions = SRR3083026_CpG_calls.keys()
for position in methyl_positions:

    for loci in sorted_gene_loci:
        gene = loci[0]
        start_bp = int(loci[1])
        end_bp = int(loci[2])
        if int(position) >= start_bp and int(position)<=end_bp:
            methyl_calls = SRR3083026_CpG_calls[position]
            methylated = methyl_calls[0]
            unmethylated = methyl_calls[1]
            SRR3083026_gene_methylcounts[gene][0] += methylated
            SRR3083026_gene_methylcounts[gene][1] += unmethylated
            break

#do the same for SRR3083029
SRR3083029_gene_methylcounts = {}
for loci in sorted_gene_loci:
    gene_name = loci[0]
    SRR3083029_gene_methylcounts[gene_name] = [0,0]


methyl_positions = SRR3083029_CpG_calls.keys()
for position in methyl_positions:

    for loci in sorted_gene_loci:
        gene = loci[0]
        start_bp = int(loci[1])
        end_bp = int(loci[2])
        if int(position) >= start_bp and int(position)<=end_bp:
            methyl_calls = SRR3083029_CpG_calls[position]
            methylated = methyl_calls[0]
            unmethylated = methyl_calls[1]
            SRR3083029_gene_methylcounts[gene][0] += methylated
            SRR3083029_gene_methylcounts[gene][1] += unmethylated
            break

#print(SRR3083026_gene_methylcounts)           
#print(SRR3083029_gene_methylcounts)

In [5]:
genes = SRR3083026_gene_methylcounts.keys()
methylated_genes = []

fold_changes = []

for gene in genes:
    #calculate percentage methylation for SRR3083026
    SRR3083026_counts = SRR3083026_gene_methylcounts[gene]
    methyl_counts = SRR3083026_counts[0]
    unmethyl_counts = SRR3083026_counts[1]
    #skip gene if methylation for E4 cell is zero
    if methyl_counts == 0:
        continue
    total_counts = methyl_counts + unmethyl_counts 
    #skip gene since no count data at all for gene
    if total_counts == 0:
        continue
    SRR3083026_pct_methyl = methyl_counts/total_counts
    
    #calculate percentage methylation for SRR3083029
    SRR3083029_counts = SRR3083029_gene_methylcounts[gene]
    methyl_counts = SRR3083029_counts[0]
    unmethyl_counts = SRR3083029_counts[1]
    total_counts = methyl_counts + unmethyl_counts 
    #skip gene since no count data at all for gene
    if total_counts == 0:
        continue
    SRR3083029_pct_methyl = methyl_counts/total_counts
    
    #calculate fold change
    fold_change = SRR3083029_pct_methyl/SRR3083026_pct_methyl
    methylated_genes.append(gene)
    fold_changes.append(fold_change)

        
print(methylated_genes)
print(fold_changes)

['Mpp6', 'Dfna5', 'Osbpl3', 'Cycs', '5430402O13Rik', '4921507P07Rik', 'Npvf', 'C530044C16Rik', 'Mir148a', 'Gm6559', 'Nfe2l3', 'Hnrnpa2b1', 'Cbx3', 'Snx10', 'Skap2', 'Halr1', 'Hotairm1', 'Hoxa2', 'Hoxaas2', 'Hoxa3', 'Hoxa7', 'Hoxa9', 'Mir196b', 'Hoxa10', 'Hoxa11', 'Hoxa11os', 'Hoxa13', 'Hottip', 'Evx1os', 'Evx1', '1700094M24Rik', 'Hibadh', 'Tax1bp1', 'Jazf1', 'Gm4872', '9430076C15Rik', 'Creb5', 'Tril', 'Cpvl', '4921529L05Rik', 'Chn2', '9130019P16Rik', 'Wipf3', 'Scrn1', 'Fkbp14', 'Plekha8', 'Mturn', 'Znrf2', 'Nod1', 'Ggct', 'Gars', 'Crhr2', 'Inmt', 'Fam188b', 'Aqp1', 'Ghrhr', '6430584L05Rik', 'Adcyap1r1', 'Neurod6', 'Gm3279', 'Ccdc129', 'Ppp1r17', 'Pde1c', 'Lsm5', 'Avl9', 'Kbtbd2', 'Fkbp9', 'Nt5c3', 'Vmn1r4', 'Vmn1r5', 'Vmn1r7', 'Vmn1r8', 'Vmn1r11', 'Vmn1r13', 'Vmn1r14', 'Vmn1r15', 'Ppm1k', 'Herc6', 'Pyurf', 'Lancl2', 'Vopp1', 'Vmn1r27', 'Vmn1r29', 'Vmn1r30', 'Abcg2', 'Herc3', 'Fam13a', 'Tigd2', 'Gprin3']
[2.849144510876479, 4.31040997543926, 4.426348831131807, 7.212121212121212, 2.86101

In [6]:
output_file = "/Users/cmdb/qbb2020-answers/assignment6/fold-methylation.txt"
with open(output_file, 'w') as f:
    f.write("gene" + '\t' + 'fold_change, E5.5/E4.0' + '\n')
    for i, gene in enumerate(methylated_genes):
        f.write(gene + '\t' + str(fold_changes[i]) + '\n')
